<a href="https://colab.research.google.com/github/sharma-learn/jaxing/blob/sharma-learn-patch-1/src/jaxing_multi_output_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [ ]:
def build_model_with_functional():
  # Define model layers.
  # train.columns
  input_layer = Input(shape=(8,))
  first_dense = Dense(units=128, activation='relu')(input_layer)
  second_dense = Dense(units=128, activation='relu')(first_dense)

  # Y1 output will be fed directly from the second dense
  y1_output_n = Dense(units=1, name='y1_output')(second_dense)
  third_dense = Dense(units=66, activation='relu')(second_dense)

  # Y2 output will come via the third dense
  y2_output_n = Dense(units=1, name='y2_output')(third_dense)

  # Define the model with the input layer and a list of output layers
  model = Model(inputs=input_layer, outputs=[y1_output_n, y2_output_n])
  return model

model = build_model_with_functional()
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          1,152 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │         16,512 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 66)             │          8,514 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ y1_output (Dense)         │ (None, 1)              │            129 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ y2_output (Dense)         │ (None, 1)              │             67 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 26,374 (103.02 KB)

 Trainable params: 26,374 (103.02 KB)

 Non-trainable params: 0 (0.00 B)

None


## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [ ]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})